In [1]:
from sae_lens import SAE, LanguageModelSAERunnerConfig, SAEConfig
from datasets import load_dataset  
from transformer_lens import HookedTransformer
import torch
import plotly.express as px

/mnt/c/Users/thijm/GitHub/Master_related/ProjectAI_related/OthelloGPT-SAELens/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# token_dataset = load_dataset('taufeeque/othellogpt', split='train')
# sae = SAE.load_from_pretrained('othello_sae')

In [3]:
# batch_tokens = token_dataset[:32]["tokens"]
# print(batch_tokens)

In [4]:
# sae.eval()
# sae.cuda()
# with torch.no_grad():
#     # activation store can give us tokens.
#     batch_tokens = token_dataset[:32]["tokens"]
#     # convert to tensor
#     batch_tokens = torch.tensor(batch_tokens)[:, -1]
#     _, cache = model.run_with_cache(batch_tokens.cuda(), prepend_bos=False)
    
    
#     feature_acts = sae.encode(cache[sae.cfg.hook_name])
#     sae_out = sae.decode(feature_acts)
    
    
#     l0 = (feature_acts[:, 1:] > 0).float().sum(-1).detach()
#     print("average l0", l0.mean().item())
#     px.histogram(l0.flatten().cpu().numpy()).show()

In [5]:
# from transformer_lens import utils
# from functools import partial

# # next we want to do a reconstruction test.
# def reconstr_hook(activation, hook, sae_out):
#     return sae_out


# def zero_abl_hook(activation, hook):
#     return torch.zeros_like(activation)


# print("Orig", model(batch_tokens, return_type="loss").item())
# print(
#     "reconstr",
#     model.run_with_hooks(
#         batch_tokens,
#         fwd_hooks=[
#             (
#                 sae.cfg.hook_name,
#                 partial(reconstr_hook, sae_out=sae_out),
#             )
#         ],
#         return_type="loss",
#     ).item(),
# )
# print(
#     "Zero",
#     model.run_with_hooks(
#         batch_tokens,
#         return_type="loss",
#         fwd_hooks=[(sae.cfg.hook_name, zero_abl_hook)],
#     ).item(),
# )

In [6]:
# create dummy tokenizer

import string

# Initialize the dictionary
vocab = {}

# Generate the keys (0 to 59) and the corresponding values (A1 to H8)
letters = string.ascii_uppercase[:8]  # Get letters 'A' to 'H'
index = 0

for letter in letters:
    for number in range(1, 9):  # Numbers 1 to 8
        vocab[index] = f"{letter}{number}"
        index += 1


In [7]:
from sae_vis.data_config_classes import SaeVisConfig
from sae_vis.data_storing_fns import SaeVisData
import sae_vis
import importlib
importlib.reload(sae_vis)
importlib.reload(sae_vis.data_storing_fns)
from sae_vis import utils_fns
importlib.reload(utils_fns)
# importlib.reload(sae_vis.data_config_classes)
from sae_vis import data_fetching_fns
importlib.reload(data_fetching_fns)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

sae = SAE.load_from_pretrained('othello_sae')


test_feature_idx_gpt = list(range(10)) + [10]
hook_name = sae.cfg.hook_name
model = HookedTransformer.from_pretrained('my-own-othello-model')
model.work_around_vocab = vocab

# batch_tokens = token_dataset[:32]["tokens"]
# convert to tensor
# batch_tokens = torch.tensor(batch_tokens)[:, -1]
    
feature_vis_config_gpt = SaeVisConfig(
    hook_point=hook_name,
    features=test_feature_idx_gpt,
    batch_size=10,
    minibatch_size_tokens=59,
    verbose=True,
)
token_dataset = load_dataset('taufeeque/othellogpt', split='train')
print(type(token_dataset))
print(token_dataset[:32]["tokens"])
# convert to tensor
token_dataset = torch.tensor(token_dataset[:32]["tokens"])[:, :-1].cuda()
print(token_dataset.shape)
# print(token_dataset[:32]["tokens"])
sae.eval()

sae_vis_data_gpt = SaeVisData.create(
    encoder=sae.cuda(),
    model=model, # type: ignore
    tokens=token_dataset,  # type: ignore
    cfg=feature_vis_config_gpt,
)

This happend
{'n_layers': 6, 'd_model': 128, 'd_mlp': 512, 'd_head': 64, 'n_heads': 8, 'n_ctx': 59, 'd_vocab': 61, 'act_fn': 'gelu', 'attn_only': False, 'normalization_type': 'LNPre'}
Loaded pretrained model my-own-othello-model into HookedTransformer
<class 'datasets.arrow_dataset.Dataset'>
[[20, 21, 34, 19, 13, 40, 47, 28, 12, 41, 35, 5, 10, 43, 3, 23, 6, 26, 42, 54, 52, 51, 59, 36, 39, 44, 11, 49, 4, 27, 32, 38, 33, 1, 25, 60, 48, 17, 50, 57, 29, 58, 55, 14, 22, 46, 30, 24, 9, 31, 16, 8, 56, 7, 45, 18, 15, 2, 37, 53], [34, 42, 27, 19, 50, 51, 52, 35, 41, 48, 49, 56, 36, 58, 11, 60, 20, 40, 57, 30, 24, 28, 59, 2, 21, 44, 22, 12, 13, 26, 4, 14, 15, 23, 5, 43, 33, 32, 10, 8, 16, 47, 55, 54, 46, 45, 37, 7, 53, 31, 39, 9, 29, 38, 25, 18, 3, 6, 17, 1], [20, 21, 22, 13, 14, 19, 5, 15, 29, 11, 16, 8, 28, 6, 27, 33, 4, 3, 18, 7, 2, 12, 41, 10, 32, 23, 9, 37, 24, 40, 39, 50, 25, 17, 59, 47, 1, 42, 51, 36, 54, 55, 35, 30, 34, 43, 52, 26, 48, 60, 56, 46, 45, 57, 38, 58, 31, 49, 44, 53], [34, 28

Forward passes to cache data for vis: 100%|██████████| 1/1 [00:06<00:00,  6.76s/it]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃ Task                                           ┃ Time  ┃ Pct % ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ (1) Initialization                             │ 0.02s │ 0.3%  │
│ (2) Forward passes to gather model activations │ 6.71s │ 87.1% │
│ (3) Computing feature acts from model acts     │ 0.02s │ 0.3%  │
│ (4) Getting data for tables                    │ 0.06s │ 0.7%  │
│ (5) Getting data for histograms                │ 0.09s │ 1.1%  │
│ (6) Getting data for sequences                 │ 0.76s │ 9.9%  │
│ (7) Getting data for quantiles                 │ 0.05s │ 0.7%  │
└────────────────────────────────────────────────┴───────┴───────┘

Extracting vis data from cached data: 100%|██████████| 11/11 [00:07<00:00,  1.42it/s]


In [8]:
COLAB = False
import webbrowser
import http.server
import platform

import socketserver
import threading
PORT = 8000
from IPython.display import IFrame, display
import subprocess


In [9]:
def display_vis_inline(filename: str, height: int = 850):
    '''
    Displays the HTML files in Colab. Uses global `PORT` variable defined in prev cell, so that each
    vis has a unique port without having to define a port within the function.
    '''
    if not COLAB:
        if os.path.isfile(filename):
            print(f"Opening {filename}")
            # Print current folder
            print(f"Current folder: {os.getcwd()}")
            # Get the absolute path
            file_path = os.path.abspath(filename)
    
            # Check if running under WSL
            if 'microsoft-standard' in platform.uname().release:
                # Use the Windows default browser to open the file
                windows_path = subprocess.check_output(['wslpath', '-w', file_path]).decode().strip()
                subprocess.run(['explorer.exe', windows_path])
            else:
                # For other environments, use the default web browser
                webbrowser.open(f"file://{file_path}")
            
            # Display inline in Jupyter Notebook
            display(IFrame(src=f"file://{file_path}", width="100%", height=height))
        else:
            print(f"File not found: {filename}")

In [10]:
for feature in test_feature_idx_gpt:
    filename = f"{feature}_feature_vis_demo_gpt.html"
    sae_vis_data_gpt.save_feature_centric_vis(filename, feature)
    display_vis_inline(filename)

Saving feature-centric vis: 100%|██████████| 11/11 [00:01<00:00,  9.90it/s]


Opening 0_feature_vis_demo_gpt.html
Current folder: /mnt/c/Users/thijm/GitHub/Master_related/ProjectAI_related/OthelloGPT-SAELens


Saving feature-centric vis: 100%|██████████| 11/11 [00:01<00:00,  9.63it/s]


Opening 1_feature_vis_demo_gpt.html
Current folder: /mnt/c/Users/thijm/GitHub/Master_related/ProjectAI_related/OthelloGPT-SAELens


Saving feature-centric vis: 100%|██████████| 11/11 [00:01<00:00,  8.49it/s]


...

Opening 8_feature_vis_demo_gpt.html
Current folder: /mnt/c/Users/thijm/GitHub/Master_related/ProjectAI_related/OthelloGPT-SAELens


Saving feature-centric vis: 100%|██████████| 11/11 [00:01<00:00,  8.70it/s]


Opening 9_feature_vis_demo_gpt.html
Current folder: /mnt/c/Users/thijm/GitHub/Master_related/ProjectAI_related/OthelloGPT-SAELens


Saving feature-centric vis: 100%|██████████| 11/11 [00:01<00:00,  9.82it/s]


Opening 10_feature_vis_demo_gpt.html
Current folder: /mnt/c/Users/thijm/GitHub/Master_related/ProjectAI_related/OthelloGPT-SAELens
